In [ ]:
%status

In [ ]:
%fund rinkeby

In [ ]:
%budget polygon 1

In [ ]:
%connect timeout=10m

Progress: 1/3
    Demand created. Waiting for counter proposal.
    Searching for just any machine...
    Will try to connect for 10 minutes.


Progress: 2/3
    Agreement created.
    Connected to KDGNode [0xc4bc7aa95302f14bc61424959961aad82034962d]
    RAM: 2.5960467755794525 GB
    DISK: 34.16630554199219 GB
    CPU: 8 cores
Progress: 3/3
    Engine is starting. It might take few minutes...


In [18]:
%upload california_housing_train.csv

Sending...
File uploaded.


In [19]:
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt

# The following lines adjust the granularity of reporting. 
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format
training_df = pd.read_csv(filepath_or_buffer="workdir/california_housing_train.csv")
# Scale the label.
training_df["median_house_value"] /= 1000.0

2023-06-21 19:02:57.370039: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [20]:
def build_model(my_learning_rate):
  """Create and compile a simple linear regression model."""
  # Most simple tf.keras models are sequential.
  model = tf.keras.models.Sequential()

  # Describe the topography of the model.
  # The topography of a simple linear regression model
  # is a single node in a single layer.
  model.add(tf.keras.layers.Dense(units=1, 
                                  input_shape=(1,)))

  # Compile the model topography into code that TensorFlow can efficiently
  # execute. Configure training to minimize the model's mean squared error. 
  model.compile(optimizer=tf.keras.optimizers.experimental.RMSprop(learning_rate=my_learning_rate),
                loss="mean_squared_error",
                metrics=[tf.keras.metrics.RootMeanSquaredError()])

  return model        


def train_model(model, df, feature, label, epochs, batch_size):
  """Train the model by feeding it data."""

  # Feed the model the feature and the label.
  # The model will train for the specified number of epochs. 
  history = model.fit(x=df[feature],
                      y=df[label],
                      batch_size=batch_size,
                      epochs=epochs)

  # Gather the trained model's weight and bias.
  trained_weight = model.get_weights()[0]
  trained_bias = model.get_weights()[1]

  # The list of epochs is stored separately from the rest of history.
  epochs = history.epoch
  
  # Isolate the error for each epoch.
  hist = pd.DataFrame(history.history)

  # To track the progression of training, we're going to take a snapshot
  # of the model's root mean squared error at each epoch. 
  rmse = hist["root_mean_squared_error"]

  return trained_weight, trained_bias, epochs, rmse

def plot_the_model(trained_weight, trained_bias, feature, label):
  """Plot the trained model against 200 random training examples."""

  # Label the axes.
  plt.xlabel(feature)
  plt.ylabel(label)

  # Create a scatter plot from 200 random points of the dataset.
  random_examples = training_df.sample(n=200)
  plt.scatter(random_examples[feature], random_examples[label])

  # Create a red line representing the model. The red line starts
  # at coordinates (x0, y0) and ends at coordinates (x1, y1).
  x0 = 0
  y0 = trained_bias
  x1 = random_examples[feature].max()
  y1 = trained_bias + (trained_weight * x1)
  plt.plot([x0, x1], [y0, y1], c='r')

  # Render the scatter plot and the red line.
  plt.show()


def plot_the_loss_curve(epochs, rmse):
  """Plot a curve of loss vs. epoch."""

  plt.figure()
  plt.xlabel("Epoch")
  plt.ylabel("Root Mean Squared Error")

  plt.plot(epochs, rmse, label="Loss")
  plt.legend()
  plt.ylim([rmse.min()*0.97, rmse.max()])
  plt.show()  

In [23]:
# The following variables are the hyperparameters.
learning_rate = 0.01
epochs = 30
batch_size = 30

# Specify the feature and the label.
my_feature = "total_rooms"  # the total number of rooms on a specific city block.
my_label="median_house_value" # the median value of a house on a specific city block.
# That is, you're going to create a model that predicts house value based 
# solely on total_rooms.  

# Discard any pre-existing version of the model.
my_model = None

# Invoke the functions.
my_model = build_model(learning_rate)
weight, bias, epochs, rmse = train_model(my_model, training_df, 
                                         my_feature, my_label,
                                         epochs, batch_size)

print("\nThe learned weight for your model is %.4f" % weight)
print("The learned bias for your model is %.4f\n" % bias )

#plot_the_model(weight, bias, my_feature, my_label)
#plot_the_loss_curve(epochs, rmse)

Epoch 1/30
567/567 [==============================] - 1s 1ms/step - loss: 215438.1719 - root_mean_squared_error: 464.1532
Epoch 2/30
567/567 [==============================] - 1s 979us/step - loss: 28140.0469 - root_mean_squared_error: 167.7500
Epoch 3/30
567/567 [==============================] - 1s 998us/step - loss: 27300.8828 - root_mean_squared_error: 165.2298
Epoch 4/30
567/567 [==============================] - 1s 1ms/step - loss: 26580.5625 - root_mean_squared_error: 163.0355
Epoch 5/30
567/567 [==============================] - 1s 988us/step - loss: 25696.9277 - root_mean_squared_error: 160.3026
Epoch 6/30
567/567 [==============================] - 1s 1ms/step - loss: 25300.5566 - root_mean_squared_error: 159.0615
Epoch 7/30
567/567 [==============================] - 1s 1ms/step - loss: 24456.3672 - root_mean_squared_error: 156.3853
Epoch 8/30
567/567 [==============================] - 1s 1ms/step - loss: 23515.6543 - root_mean_squared_error: 153.3481
Epoch 9/30
567/567 [=====

In [24]:
def predict_house_values(n, feature, label):
  """Predict house values based on a feature."""

  batch = training_df[feature][10000:10000 + n]
  predicted_values = my_model.predict_on_batch(x=batch)

  print("feature   label          predicted")
  print("  value   value          value")
  print("          in thousand$   in thousand$")
  print("--------------------------------------")
  for i in range(n):
    print ("%5.0f %6.0f %15.0f" % (training_df[feature][10000 + i],
                                   training_df[label][10000 + i],
                                   predicted_values[i][0] ))

In [25]:
predict_house_values(10, my_feature, my_label)

feature   label          predicted
  value   value          value
          in thousand$   in thousand$
--------------------------------------
 1960     53             152
 3400     92             167
 3677     69             170
 2202     62             155
 2403     80             157
 5652    295             191
 3318    500             167
 2552    342             158
 1364    118             146
 3468    128             168
2023-06-21 19:07:07.236938: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [10,1]
	 [[{{node Placeholder/_0}}]]


In [26]:
%disconnect

Disconnecting... done.
Connection time: 6 minutes
Total cost: 0.005624961037186111
Allocated 0.991143181167029444 glm on polygon